# 1. Partial Variable


## 1.1 原始用法

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

msgs = [
    ("system", "你是一个有氧拳击教练，请帮我解释拳击术语，如果术语是英文，请先翻译为中文。"
               "{format_instructions}"),
    ("human", "术语：{name}")
]

format_instructions = """结果请按以下形式输出为 json 格式:

{
	"terminology": string  // 中文术语名称
	"explanation": string  // 术语的解释
}
"""

prompt = ChatPromptTemplate.from_messages(msgs).partial(format_instructions=format_instructions)

chat_model = ChatOpenAI(model_name="openai/gpt-4o-mini")
chain = prompt | chat_model

resp = chain.invoke({"name": "下潜躲避"})
print(resp.content)


{
	"terminology": "下潜躲避",
	"explanation": "下潜躲避是一种拳击技术，指的是拳击手通过弯曲膝盖和下沉身体来躲避对手的攻击。这个动作可以帮助拳击手降低重心，从而避免被击中，同时也为反击创造机会。"
}


## 1.2 配合 OutputParser 规范输出

In [22]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


class Work(BaseModel):
    title: str = Field(description="作品名称")
    description: str = Field(description="作品简介")


parser = JsonOutputParser(pydantic_object=Work)

prompt = PromptTemplate(
    template="请列举 3 部 {author} 的作品。\n{format_instructions}",
    input_variables=["author"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chat = ChatOpenAI(model_name="openai/gpt-4o-mini")
chain = prompt | chat | parser

resp = chain.invoke({"author": "鲁迅"})
print(resp, "\n\n", type(resp), "\n", len(resp), "\n", type(resp[0]))


[{'title': '狂人日记', 'description': '这是鲁迅的第一篇白话文小说，通过一个精神病患者的视角，揭示了社会的黑暗与人性的扭曲。'}, {'title': '阿Q正传', 'description': '这部作品通过阿Q这一角色，讽刺了中国社会的劣根性，深刻反映了那个时代人们的精神状态。'}, {'title': '呐喊', 'description': '《呐喊》是鲁迅的短篇小说集，集中展现了社会的压迫和个人的无奈，是中国现代文学的重要作品。'}] 

 <class 'list'> 
 3 
 <class 'dict'>


# 2. Few Shot Prompt Template

核心用法是，创建一个 `FewShotPromptTemplate` 类型的对象，完成创建后，这个 prompt 可以传给 ChatModel 或 LLM 直接使用。

传入参数为：

- example：few shot 的样例数据
- example_prompt：PromptTemplate类型，要与 example 能对应上
- prefix 和 suffix：本 prompt 的前缀和后缀，也就是说，通过上面两个参数，这个 prompt 已经有了大体内容，但还不完整，然后可以通过这两个参数把 prompt 的完整内容补齐，其中，suffix 是必传的

另外还有个参数是 `example_selector`，它和 example 的作用是一样的，都是为了提供 Few Shot 样例数据，二选一即可。二者的主要区别是，如果样例数据很多，提示词了放不下，或者用户提供了很多样例数据，需要大模型根据语境动态选择一部分样例时，使用这个参数。

`example_selector` 的使用，下面会单独一节说明，本节从 Fixed Example 讲起。

## 2.1 Fixed Examples

In [36]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="openai/gpt-4o-mini")

samples = [
    {
        "flower_type": "玫瑰",
        "occasion": "爱情",
        "ad_copy": "玫瑰，浪漫的象征，是你向心爱的人表达爱意的最佳选择。"
    },
    {
        "flower_type": "康乃馨",
        "occasion": "母亲节",
        "ad_copy": "康乃馨代表着母爱的纯洁与伟大，是母亲节赠送给母亲的完美礼物。"
    },
    {
        "flower_type": "百合",
        "occasion": "庆祝",
        "ad_copy": "百合象征着纯洁与高雅，是你庆祝特殊时刻的理想选择。"
    },
    {
        "flower_type": "向日葵",
        "occasion": "鼓励",
        "ad_copy": "向日葵象征着坚韧和乐观，是你鼓励亲朋好友的最好方式。"
    }
]

example_template = "鲜花类型: {flower_type}\n场合: {occasion}\n文案: {ad_copy}"
example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=samples,
    suffix="鲜花类型: {flower_type}\n场合: {occasion}",
)
res = prompt.invoke({"flower_type": "桂花", "occasion": "中秋"})
print(res.to_string(), "\n\n")

# print(chat.invoke(res).content)


鲜花类型: 玫瑰
场合: 爱情
文案: 玫瑰，浪漫的象征，是你向心爱的人表达爱意的最佳选择。

鲜花类型: 康乃馨
场合: 母亲节
文案: 康乃馨代表着母爱的纯洁与伟大，是母亲节赠送给母亲的完美礼物。

鲜花类型: 百合
场合: 庆祝
文案: 百合象征着纯洁与高雅，是你庆祝特殊时刻的理想选择。

鲜花类型: 向日葵
场合: 鼓励
文案: 向日葵象征着坚韧和乐观，是你鼓励亲朋好友的最好方式。

鲜花类型: 桂花
场合: 中秋 




In [37]:
print(chat.invoke(res).content)

文案: 桂花散发着浓郁的香气，象征着团圆与美好，是中秋佳节送给亲友的温馨祝福。


In [32]:
template = """请根据下面的鲜花类型和场合，写一段合适的文案，用于宣传推广。
鲜花类型：{flower_type}
场合：{occasion}
"""

prompt2 = FewShotPromptTemplate(
    prefix=template,
    example_prompt=example_prompt,
    examples=samples,
    input_variables=["flower_type", "occasion"],
    suffix=""
)

res = prompt2.invoke({"flower_type": "桂花", "occasion": "中秋"})
print(res.to_string())

请根据下面的鲜花类型和场合，写一段合适的文案，用于宣传推广。
鲜花类型：桂花
场合：中秋


鲜花类型: 玫瑰
场合: 爱情
文案: 玫瑰，浪漫的象征，是你向心爱的人表达爱意的最佳选择。

鲜花类型: 康乃馨
场合: 母亲节
文案: 康乃馨代表着母爱的纯洁与伟大，是母亲节赠送给母亲的完美礼物。

鲜花类型: 百合
场合: 庆祝
文案: 百合象征着纯洁与高雅，是你庆祝特殊时刻的理想选择。

鲜花类型: 向日葵
场合: 鼓励
文案: 向日葵象征着坚韧和乐观，是你鼓励亲朋好友的最好方式。


In [33]:
print(chat.invoke(res).content)

鲜花类型：桂花  
场合：中秋  
文案：桂花，香气四溢，象征着团圆与美好，是中秋佳节送给亲友的心意之选。让这份浓郁的香气，伴随明月，共同传递家人间的深情厚谊，带来无尽的祝福与温馨。


In [47]:
template = """请根据下面的鲜花类型和场合，写一段合适的文案，用于宣传推广。
鲜花类型：{flower_type}
场合：{occasion}
"""

prompt3 = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=samples,
    prefix=template,
    suffix="输出要求：只要输出文案即可"
)

res = prompt3.invoke({"flower_type": "桂花", "occasion": "中秋"})
print(res.to_string(), "\n\n\n")
print(chat.invoke(res).content)

请根据下面的鲜花类型和场合，写一段合适的文案，用于宣传推广。
鲜花类型：桂花
场合：中秋


鲜花类型: 玫瑰
场合: 爱情
文案: 玫瑰，浪漫的象征，是你向心爱的人表达爱意的最佳选择。

鲜花类型: 康乃馨
场合: 母亲节
文案: 康乃馨代表着母爱的纯洁与伟大，是母亲节赠送给母亲的完美礼物。

鲜花类型: 百合
场合: 庆祝
文案: 百合象征着纯洁与高雅，是你庆祝特殊时刻的理想选择。

鲜花类型: 向日葵
场合: 鼓励
文案: 向日葵象征着坚韧和乐观，是你鼓励亲朋好友的最好方式。

输出要求：只要输出文案即可 



桂花，馨香四溢，象征着团圆与美好，是中秋佳节送给亲友的绝佳选择。让这份芬芳伴随月光，传递你对家人和朋友的思念与祝福，共享团圆的温暖时刻。


In [48]:
template = """请根据下面的鲜花类型和场合，写一段合适的文案，用于宣传推广。
鲜花类型：{flower}
场合：{case}
"""

prompt4 = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=samples,
    input_variables=["flower", "case"],
    suffix=template
)

res = prompt4.invoke({"flower": "桂花", "case": "中秋"})
print(res.to_string(), "\n\n\n")
print(chat.invoke(res).content)

鲜花类型: 玫瑰
场合: 爱情
文案: 玫瑰，浪漫的象征，是你向心爱的人表达爱意的最佳选择。

鲜花类型: 康乃馨
场合: 母亲节
文案: 康乃馨代表着母爱的纯洁与伟大，是母亲节赠送给母亲的完美礼物。

鲜花类型: 百合
场合: 庆祝
文案: 百合象征着纯洁与高雅，是你庆祝特殊时刻的理想选择。

鲜花类型: 向日葵
场合: 鼓励
文案: 向日葵象征着坚韧和乐观，是你鼓励亲朋好友的最好方式。

请根据下面的鲜花类型和场合，写一段合适的文案，用于宣传推广。
鲜花类型：桂花
场合：中秋
 



鲜花类型: 桂花  
场合: 中秋  
文案: 桂花，象征着团圆与思念，是中秋佳节传递情感的最佳选择。在这个象征着丰收与团聚的时刻，送上一束桂花，表达对亲人和朋友的祝福，让浓郁的桂香伴随明月，共同庆祝美好的团圆时光。


## 2.2 使用示例选择器

In [17]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_qdrant import QdrantVectorStore
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaEmbeddings

chat = ChatOpenAI(model_name="openai/gpt-4o-mini")
embedding = OllamaEmbeddings(model="nomic-embed-text", base_url="http://localhost:11434")

samples = [
    {
        "flower_type": "玫瑰",
        "occasion": "爱情",
        "ad_copy": "玫瑰，浪漫的象征，是你向心爱的人表达爱意的最佳选择。"
    },
    {
        "flower_type": "康乃馨",
        "occasion": "母亲节",
        "ad_copy": "康乃馨代表着母爱的纯洁与伟大，是母亲节赠送给母亲的完美礼物。"
    },
    {
        "flower_type": "百合",
        "occasion": "庆祝",
        "ad_copy": "百合象征着纯洁与高雅，是你庆祝特殊时刻的理想选择。"
    },
    {
        "flower_type": "向日葵",
        "occasion": "鼓励",
        "ad_copy": "向日葵象征着坚韧和乐观，是你鼓励亲朋好友的最好方式。"
    }
]

template = "鲜花类型: {flower_type}\n场合: {occasion}\n文案: {ad_copy}"
example_selector = SemanticSimilarityExampleSelector.from_examples(samples, embedding, QdrantVectorStore, k=1)

prompt = FewShotPromptTemplate(
    example_prompt=PromptTemplate.from_template(template),
    example_selector=example_selector,
    suffix="鲜花类型: {flower_type}\n场合: {occasion}",
)
res = prompt.invoke({"flower_type": "野玫瑰", "occasion": "爱情"})

print(res.to_string(), "\n\n\n")

print(chat.invoke(res).content)

鲜花类型: 玫瑰
场合: 爱情
文案: 玫瑰，浪漫的象征，是你向心爱的人表达爱意的最佳选择。

鲜花类型: 野玫瑰
场合: 爱情 



文案: 野玫瑰，虽不如红玫瑰般夺目，却散发着自然的纯真与坚韧，寓意着真实而不拘一格的爱情。送上一束野玫瑰，让心爱的人感受到你对她独特而深沉的爱意。


In [19]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

example_selector = SemanticSimilarityExampleSelector.from_examples(samples, embedding, QdrantVectorStore, k=1)

prompt = FewShotPromptTemplate(
    example_prompt=PromptTemplate.from_template(template),
    example_selector=example_selector,
    suffix="鲜花类型: {flower_type}\n场合: {occasion}",
)
res = prompt.invoke({"flower_type": "野玫瑰", "occasion": "爱情"})

print(res.to_string(), "\n\n\n")

print(chat.invoke(res).content)

鲜花类型: 康乃馨
场合: 母亲节
文案: 康乃馨代表着母爱的纯洁与伟大，是母亲节赠送给母亲的完美礼物。

鲜花类型: 野玫瑰
场合: 爱情 



文案: 野玫瑰象征着热烈与自由，传达出真挚的爱意，是表达爱情的最佳选择。在这个特别的时刻，让野玫瑰为你的心声添上浪漫的色彩。


# 3. Few Shot ChatMessage Prompt Template

把 Few Shot 用于对话，需要使用 `FewShotChatMessagePromptTemplate`，其使用思路上与 `FewShotPromptTemplate` 是类似的，但具体调用起来相反。主要区别包括：

1. 传递给大模型的，还是 `ChatPromptTemplate`，该模板仅提供了一种新类型的 Message（与 AIMessage，HumanMessage 等是对等的），其的作用，仅相当 `FewShotPromptTemplate` 中 `example_prompt` 的值
2. 该模板会也会生成类似下面这种聊天对话记录形式的样本，插入到聊天对话中去
```python
[
    HumanMessage(content=""),
    AIMessage(content=""),
    HumanMessage(content=""),
    AIMessage(content=""),
    ...
]
```

## 3.1 Fixed Examples

In [4]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {"input": "2 🦜 2", "output": "4"},
    {"input": "2 🦜 3", "output": "5"},
]

few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=ChatPromptTemplate.from_messages(
        [
            ("human", "{input}"),
            ("ai", "{output}"),
        ]
    )
)

print(few_shot_prompt.invoke({}).to_messages(), "\n\n")

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{inp}"),
    ]
)

res = final_prompt.invoke({"inp": "21 🦜 32"})
print(res.to_messages(), "\n\n")

from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="openai/gpt-4o-mini")

resp = chat.invoke(res)
print(resp.content)


[HumanMessage(content='2 🦜 2', additional_kwargs={}, response_metadata={}), AIMessage(content='4', additional_kwargs={}, response_metadata={}), HumanMessage(content='2 🦜 3', additional_kwargs={}, response_metadata={}), AIMessage(content='5', additional_kwargs={}, response_metadata={})] 


[SystemMessage(content='You are a wondrous wizard of math.', additional_kwargs={}, response_metadata={}), HumanMessage(content='2 🦜 2', additional_kwargs={}, response_metadata={}), AIMessage(content='4', additional_kwargs={}, response_metadata={}), HumanMessage(content='2 🦜 3', additional_kwargs={}, response_metadata={}), AIMessage(content='5', additional_kwargs={}, response_metadata={}), HumanMessage(content='21 🦜 32', additional_kwargs={}, response_metadata={})] 


53


## 3.2 Dynamic FewShot

也是用到 `example_selector`，与前面用法一致。

In [24]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_qdrant import QdrantVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

examples = [
    {"input": "2 🦜 2", "output": "4"},
    {"input": "2 🦜 3", "output": "5"},
    {"input": "2 🦜 4", "output": "6"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the moon",
        "output": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
example_selector = SemanticSimilarityExampleSelector.from_examples(examples, embedding, QdrantVectorStore, k=2,
                                                                   location=":memory:"  # **vectorstore_cls_kwargs
                                                                   )

res = example_selector.select_examples({"input": "dog"})
print(res, "\n\n")

few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    # input_variables=["input"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages([
        ("human", "{input}"),
        ("ai", "{output}"),
    ]),
)

print(few_shot_prompt.invoke({"input":"What's 3 🦜 3?"}).to_messages(), "\n\n")

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{inp}"),
    ]
)

res = final_prompt.invoke({"inp": "21 🦜 32"})
print(res.to_messages(), "\n\n")

from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="openai/gpt-4o-mini")

resp = chat.invoke(res)
print(resp.content)


[{'input': 'What did the cow say to the moon?', 'output': 'nothing at all', '_id': 'ca1c219c634d439fb8c1aa05b05aafd8', '_collection_name': '7464f0f8117f4128acf03a6d1ba400cb'}, {'input': '2 🦜 4', 'output': '6', '_id': 'aa3c49aaf56a4a32a9438328103bb63a', '_collection_name': '7464f0f8117f4128acf03a6d1ba400cb'}] 


[HumanMessage(content='2 🦜 4', additional_kwargs={}, response_metadata={}), AIMessage(content='6', additional_kwargs={}, response_metadata={}), HumanMessage(content='2 🦜 3', additional_kwargs={}, response_metadata={}), AIMessage(content='5', additional_kwargs={}, response_metadata={})] 


[SystemMessage(content='You are a wondrous wizard of math.', additional_kwargs={}, response_metadata={}), HumanMessage(content='2 🦜 3', additional_kwargs={}, response_metadata={}), AIMessage(content='5', additional_kwargs={}, response_metadata={}), HumanMessage(content='2 🦜 4', additional_kwargs={}, response_metadata={}), AIMessage(content='6', additional_kwargs={}, response_metadata={}), Human